In [ ]:
# Step 1: Setup Environment (Install required packages)
! pip install ultralytics roboflow labelImg

In [18]:
# Step 2: Download Stop Sign Dataset from Roboflow
! pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9aDEd1JT9IyNxINsqkRf")
project = rf.workspace("cv-1dnsy").project("stop-signs-5wqsj")
version = project.version(11)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# <class_id> <x_center> <y_center> <width> <height>
# x_center_pixel = x_center * image_width
# y_center_pixel = y_center * image_height
# width_pixel = width * image_width
# height_pixel = height * image_height

# Total Images == 540
# train == 495
# valid == 22
# test ==  23

In [ ]:
# Step3: Split Dataset in train, test and valid
import os
import shutil
import random
val_images_dir = '/data/praveen/praveen_pole_detection/Stop-Signs-11/valid/images'
val_labels_dir = '/data/praveen/praveen_pole_detection/Stop-Signs-11/valid/labels'
test_images_dir = '/data/praveen/praveen_pole_detection/Stop-Signs-11/test/images'
test_labels_dir = '/data/praveen/praveen_pole_detection/Stop-Signs-11/test/labels'

os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

val_images = os.listdir(val_images_dir)
random.shuffle(val_images)

split_index = len(val_images) // 2
test_subset = val_images[split_index:]

for img_file in test_subset:
    label_file = img_file.rsplit('.', 1)[0] + '.txt'
    shutil.move(os.path.join(val_images_dir, img_file), os.path.join(test_images_dir, img_file))
    shutil.move(os.path.join(val_labels_dir, label_file), os.path.join(test_labels_dir, label_file))

In [1]:
print(len(os.listdir('/data/praveen/praveen_pole_detection/Stop-Signs-11/train/images')))
print(len(os.listdir('/data/praveen/praveen_pole_detection/Stop-Signs-11/valid/images')))
print(len(os.listdir('/data/praveen/praveen_pole_detection/Stop-Signs-11/test/images')))

495
11
34


In [1]:
## Step: Annotate Additional Images (Optional)
# labelImg
## Open your image folder (e.g., stop-signs-5wqsj/train/images).
## Set format to YOLO.
## Draw bounding boxes for pole and stop_sign.
## Save .txt files in labels/train.

# # Step: Verify data.yaml
# train: ../train/images
# val: ../valid/images
# test: ../test/images
# nc: 2
# names: ['stop_sign', 'pole']

# Step 4: Train YOLOv8 Model
from ultralytics import YOLO

# Load base model
model = YOLO("yolov8s.pt")  # You can use yolov8n.pt for faster training
# Train
model.train(
    data="/data/praveen/praveen_pole_detection/Stop-Signs-11/data.yaml",
    epochs=50,
    imgsz=640,    #YOLOv8 resizes all images to this size (640,640) before feeding them into the model.
    batch=16,     # This sets the batch size to 16 images per training step.
    name="stop_pole_detector"
)



New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.193 🚀 Python-3.10.0 torch-2.8.0+cu128 CPU (Intel Xeon Platinum 8259CL 2.50GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/data/praveen/praveen_pole_detection/Stop-Signs-11/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=stop_pol

/home/ubuntu/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768,

/home/ubuntu/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to /data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50         0G      1.271      1.825      1.175         60        640: 100% ━━━━━━━━━━━━ 31/31 0.2it/s 2:575.1ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.8it/s 1.3s
                   all         11         25      0.716       0.68      0.694      0.479

      Epoch    GPU

/home/ubuntu/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


      41/50         0G     0.9117     0.5581     0.9716         40        640: 100% ━━━━━━━━━━━━ 31/31 0.2it/s 3:126.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.7it/s 1.4s
                   all         11         25      0.791      0.769      0.812      0.646

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      42/50         0G      0.859     0.5411     0.9554         42        640: 100% ━━━━━━━━━━━━ 31/31 0.2it/s 3:146.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.7it/s 1.4s
                   all         11         25      0.837      0.846      0.895      0.667

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      43/50         0G     0.8756     0.5464     0.9484         37        640: 100% ━━━━━━━━━━━━ 31/31 0.2it/s 3:186.1ss
                 Class     Images  Instances      B

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4260364880>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [2]:
# Step 5: Run Inference
# Load trained model
model = YOLO("/data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12/weights/best.pt")

# Predict on an image
results = model.predict(source="/data/praveen/praveen_pole_detection/Stop-Signs-11/test/images/10_jpg.rf.52366b9548e51dc25fb295ed3707dc87.jpg", save=True, conf=0.25)

# Show results
results[0].show()


image 1/1 /data/praveen/praveen_pole_detection/Stop-Signs-11/test/images/10_jpg.rf.52366b9548e51dc25fb295ed3707dc87.jpg: 640x640 1 Pole, 1 Sign, 139.3ms
Speed: 8.1ms preprocess, 139.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /data/praveen/praveen_pole_detection/runs/detect/predict4


display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/432.


In [3]:
# Fixed Step 6: Proper Model Evaluation
from ultralytics import YOLO
import os

# Use YOLO's built-in validation method for proper evaluation
model = YOLO('/data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12/weights/best.pt')

# Run validation on test set - this gives proper metrics
print("Running validation on test dataset...")
metrics = model.val(data="/data/praveen/praveen_pole_detection/Stop-Signs-11/data.yaml", split='test')

# Print the metrics
print(f"\nModel Performance Metrics:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

# Also print per-class metrics if available
if hasattr(metrics.box, 'mp_per_class'):
    print(f"\nPer-class Precision: {metrics.box.mp_per_class}")
    print(f"Per-class Recall: {metrics.box.mr_per_class}")
    print(f"Per-class mAP50: {metrics.box.map50_per_class}")

Running validation on test dataset...
Ultralytics 8.3.193 🚀 Python-3.10.0 torch-2.8.0+cu128 CPU (Intel Xeon Platinum 8259CL 2.50GHz)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1928.8±380.8 MB/s, size: 81.7 KB)
val: Scanning /data/praveen/praveen_pole_detection/Stop-Signs-11/test/labels.cache... 34 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 34/34 80.5Kit/s 0.0s


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.7it/s 4.4s3.8ss
                   all         34         77      0.937      0.859      0.906      0.649
                  Pole         34         40      0.893        0.8       0.86      0.459
                  Sign         34         37      0.982      0.919      0.953      0.838
Speed: 1.1ms preprocess, 115.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /data/praveen/praveen_pole_detection/runs/detect/val5

Model Performance Metrics:
mAP50: 0.906
mAP50-95: 0.649
Precision: 0.937
Recall: 0.859


In [4]:
from ultralytics import YOLO
import cv2
import os

# Load your trained model
model = YOLO('/data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12/weights/best.pt')

# Check if video file exists
video_path = "/data/praveen/out_test/simbolic_alerts/all/e7bb03bf-967b-4dea-963f-03860c7d76ac/outwardVideo.mp4"  # Replace with your actual video file path
if not os.path.exists(video_path):
    print(f"Error: Video file '{video_path}' not found.")
    print("Please provide a valid video file path.")
else:
    # Load video
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video file")
    else:
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        print(f"Video properties: {width}x{height}, {fps} FPS")
        
        # Output video writer
        output_path = "output_video.mp4"
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        
        # Process each frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
        
            # Run YOLOv8 inference
            results = model.predict(source=frame, conf=0.25, verbose=False)
        
            # Annotate frame
            annotated_frame = results[0].plot()
        
            # Write to output video
            out.write(annotated_frame)
            
            frame_count += 1
            if frame_count % 30 == 0:  # Print progress every 30 frames
                print(f"Processed {frame_count} frames...")
        
            # Remove display code for headless environment
            # cv2.imshow("YOLOv8 Detection", annotated_frame)
            # if cv2.waitKey(1) & 0xFF == ord("q"):
            #     break
        
        # Release resources
        cap.release()
        out.release()
        # cv2.destroyAllWindows()  # Commented out for headless environment
        
        print(f"Video processing completed! Output saved as '{output_path}'")
        print(f"Total frames processed: {frame_count}")

Video properties: 1920x1080, 29.999648716057187 FPS
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...
Processed 120 frames...
Processed 150 frames...
Processed 180 frames...
Processed 210 frames...
Processed 240 frames...
Processed 270 frames...
Processed 300 frames...
Processed 330 frames...
Processed 360 frames...
Processed 390 frames...
Processed 420 frames...
Processed 450 frames...
Processed 480 frames...
Processed 510 frames...
Processed 540 frames...
Processed 570 frames...
Processed 600 frames...
Processed 630 frames...
Processed 660 frames...
Processed 690 frames...
Processed 720 frames...
Processed 750 frames...
Processed 780 frames...
Processed 810 frames...
Processed 840 frames...
Video processing completed! Output saved as 'output_video.mp4'
Total frames processed: 854


In [5]:
from ultralytics import YOLO
import cv2
import os

# Load trained YOLOv8 model
model = YOLO('/data/praveen/praveen_pole_detection/runs/detect/stop_pole_detector12/weights/best.pt')

# Load video
video_path = "/data/praveen/out_test/simbolic_alerts/all/e7bb03bf-967b-4dea-963f-03860c7d76ac/outwardVideo.mp4"   # Replace with your video file
cap = cv2.VideoCapture(video_path)

# Create output directory
output_dir = "annotated_frames"
os.makedirs(output_dir, exist_ok=True)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    results = model.predict(source=frame, conf=0.25)

    # Annotate frame
    annotated_frame = results[0].plot()

    # Save frame
    frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, annotated_frame)

    frame_count += 1

cap.release()
print(f"Saved {frame_count} frames to '{output_dir}'")


0: 384x640 (no detections), 74.8ms
Speed: 1.8ms preprocess, 74.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 (no detections), 80.1ms
Speed: 11.9ms preprocess, 80.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 76.3ms
Speed: 11.7ms preprocess, 76.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 77.9ms
Speed: 11.8ms preprocess, 77.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 76.4ms
Speed: 12.6ms preprocess, 76.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 75.6ms
Speed: 16.3ms preprocess, 75.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 76.3ms
Speed: 13.0ms preprocess, 76.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pole, 75.2ms
Speed: 12.8ms preprocess, 75.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.2ms
Speed: 13.2ms preprocess, 74.2ms inf